# <font color='#eb3483'> Feature Engineering </font>

One of the most important steps in the machine learning pipeline is engineering features - it's often the determining factor in whether you'll get a successful model! Feature engineering is the process of making new features in your dataset that better represent the problem you're trying to model. In this module we won't be exploring any new packages or skills, but will try to highlight the importance of taking the time to craft useful features when you're approaching a machine learning model.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

### <font color='#eb3483'> Why bother feature engineering? </font>
In most problems, the data we're given is messy (hence why we need to do data processing), and might not be in the format most conducive for learning. Let's consider a simple example - predicting the area of a circle. The original data we're given is the radius of each circle, and we want to build a linear regression to predict the area.

In [ ]:
area = pd.DataFrame({ 'radius': np.arange(10), 'area': 3.14* np.arange(10)**2 })
area.head()

Let's try building the model with the data as is:

In [ ]:
predictor = LinearRegression()
mse = cross_val_score(predictor, area.drop('area',axis=1),
                area['area'], scoring="neg_mean_squared_error", 
                cv=3).mean()

print("MSE :",-mse)

Yikes that's a terrible mean squared error for a simple problem. What if we engineered a new feature to be the radius^2 instead of the radius?

In [ ]:
area['radius_sq'] = area['radius']**2
predictor = LinearRegression()
mse = cross_val_score(predictor, area.drop('area',axis=1),
                area['area'], scoring="neg_mean_squared_error", 
                cv=3).mean()
print("MSE :",-round(mse,2))

Now we have a perfect prediction! In this example, all we had to do was engineer a feature that was more in-line with the problem we were trying to model. 

For more complicated models (i.e Neural Networks) some of the feature engineering can be done directly in the model (i.e. in a multi-layer neural network, the internal layers can act as learned representation of the data you're feeding in), but they require more training and are more prone to over-fitting. In general, feature engineering can lead to big boosts in predictive power with relatively little work on your end - so it's always a great place to start!

### <font color='#eb3483'> Feature Engineering Walkthrough </font>

For this section we are going to use the [Ames Housing dataset](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) which is an updated and expanded version of the Boston Housing Dataset.This [link](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) has the data dictionary.

In [ ]:
ames = pd.read_csv("data/ames.csv").drop(columns="PID").sample(500, random_state=42)
ames.columns

In [ ]:
ames.shape

In [ ]:
ames.head()

Let's take a peak at all our datatypes.

In [ ]:
ames.dtypes.head()

## <font color='#eb3483'> Data Processing </font>

Looks like we have a lot of ordinal (i.e. data that has ordered categories) and categorical data (i.e. data that has categories) We are going to replace the ordinal and categorical variables, using `mlxtend`.

In [ ]:
#Remember target is what we're trying to predict
target = "SalePrice"
#Independent variables are things we're using to try to predict it
independent_variables = ames.drop(columns=target).columns

In [ ]:
numerical_cols = ames[independent_variables].select_dtypes(np.number).columns
categorical_cols = ames.select_dtypes(exclude=np.number).columns

#Let's make an ordered mapping of all our ordinal data (i.e. values on the right are better)
ordinal_var_dict = {'LotShape': ['IR3', 'IR2', 'IR1', 'Reg'],
 'Utilities': ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
 'LandSlope': ['Sev', 'Mod', 'Gtl'],
 'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'BsmtQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'BsmtCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'BsmtExposure': ['NA', 'No', 'Mn', 'Av', 'Gd'],
 'BsmtFinType1': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
 'BsmtFinType2': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
 'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'Functional': ['Sal', 'Sev', 'Maj2', 'Maj1', 'Min2', 'Min1', 'Typ'],
 'FireplaceQu': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'GarageFinish': ['NA', 'Unf', 'RFn', 'Fin'],
 'GarageQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'GarageCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 'PavedDrive': ['N', 'P', 'Y'],
 'PoolQC': ['NA', 'Fa', 'TA', 'Gd', 'Ex'],
 'Fence': ['NA', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']}


#Let's keep track of all our ordinal and categorical data
ordinal_cols = list(ordinal_var_dict.keys())
categorical_cols = list(set(categorical_cols) - set(ordinal_cols))

### <font color='#eb3483'> Numerical Data </font>
For numerical data, we're going to do a two-step process *impute* our missing values using the median, and then *normalize* our columns (i.e. subtract the mean and divide by the standard deviation). We're going to use built-in functions from sklearn. Check them out using `?`.

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer

SimpleImputer?

Let's check out the imputer first. We'll look at the LotFrontage column (which has 83 missing values) and see what happens when we impute the data

In [ ]:
ames[numerical_cols[1]]

In [ ]:
#Simple imputer takes a strategy (i.e. replace missing values with the median value)
#And has a fit_transform function which takes a dataframe and returns a numpy array with the data and no missing vals
imputed = SimpleImputer(strategy="median").fit_transform(ames[numerical_cols])

#Notice our LotFrontage data now has no missing values!
imputed[:10,1] #we'll just look at the first 10 rows

Now let's see how to normalize. For that we'll use sklearns normalize function. Same idea we feed in a dataframe or numpy matrix and it'll normalize all of our columns and return a numpy matrix. Note that we can't feed in our raw data (it'll throw an error if there's missing values) which is we'll use the imputed data

In [ ]:
normalize(pd.DataFrame(imputed))

Now let's package it together into one beautiful extended line of code

In [ ]:
numerical_data_imputed_normalized = pd.DataFrame(
    #We're created a new dataframe where our columns have been imputed and normalized
    normalize(SimpleImputer(strategy="median").fit_transform(ames[numerical_cols])),
    columns=numerical_cols
)

### <font color='#eb3483'> Categorical Variables </font>

For categorical data we're going to use [1-hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). Which means that each category will have a binary column (i.e. if the column was gender we'd have one column for male and female and having a 1 for male means the person is male). This is super common in machine learning, and pandas even has a function for it called `get_dummies` (check out the help docs)

In [ ]:
categorical_data_dummy = pd.get_dummies(ames[categorical_cols], drop_first=True)

In [ ]:
categorical_data_dummy.head()

### <font color='#eb3483'>Ordinal variables </font>

Checking the [data dictionary](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) there are many ordinal variables (measuring quality levels of different aspects in the houses from worst to best). As a reminder ordinal data means that there are categories (like categorical) but there's an ordering to them (i.e. one category is better than the other). To represent ordinal data we want to convert it numeric values that preserve that ordering. To do that we'll use pandas built-in functionality for categorical data. The high level steps are
- For each column we'll convert it to categorical data (which means each string value will have an associated number i.e. 1 = 'Male', 2 = 'Female')
- We'll set the ordering of the categories to be what we have in our dictionary (i.e. so the 'worst' category is first, best is last)
- Then we'll set our column to just use the underlying category numbers which now preserve the order we want

In [ ]:
ordinal_data = ames[ordinal_cols]

In [ ]:
#We're going to iterate through the ordinal columns and fix them
for col_ordinal, values in ordinal_var_dict.items():
    ordinal_data[col_ordinal] = (
    ordinal_data[col_ordinal] #first let's grab all our column's data
    .astype("category") #Convert it to category type
    #for the category we're going to set the ordering of the possible values to be what we have in our ordinal_dict
    .cat.set_categories(values) 
    #This will make sure we're using the category numbers (which will be in the order we want)
    .cat.codes
)

In [ ]:
ordinal_data.head()

We join the 3 datasets

In [ ]:
ames_processed = pd.concat([
    numerical_data_imputed_normalized.reset_index(drop=True),
    categorical_data_dummy.reset_index(drop=True),
    ordinal_data.reset_index(drop=True)
], axis=1)

In [ ]:
ames_processed.shape

In [ ]:
ames_processed.head()

And just like that we have a beautiful feature engineered dataset! We've covered some standard tools but remember it's important to think about what new features are suited to the problem (i.e. is yearbuilt important or do we want to just bin it into categories for old or new house?). That's where the creative part of data science comes in, and why having domain expertise or understanding where your data is coming from is so important!